# 전처리

1. 원데이터(train.json)을 train, val 로 나누어주어야 함
2. 원데이터의 answer_start는 글자기준으로, 정답이 몇번째 글자에 있는지 알려주는것
3. 일단, answer_start + 정답글자수로 정답의 끝 글자를 추가한다.(train_answers['answer_end'])
4. 사용하려는 모델의 토크나이저를 import 하고, 토크나이징을 한다. (train_encoding)
5. transformers 라이브러리의 토크나이저를 불러와 인코딩을 했으므로, 자동으로 cls,sep 토큰이 붙고,  input_ids, token_type_ids, attention_mask이 자동으로 생성된다!
6. 하지만, 아직 answer_start와 answer_end는 토큰화 하기전 글자수 기준 숫자이다. 때문에 이거를 5에서 만들었던 '[CLS] question tokens, [sep] context tokens [sep]' 요 형태에서 start와 end 가 몇번째인지 다시 세어주어야 한다. 
7. 그것이 add_token_positions 함수 부분
8. 다 준비가 되었으니 모델에 넣을 수 있게 Dataset 형태로 변환한다.(SquadDataset)

전처리 끝!

9. 모델 학습

고쳐야 할 것
add_token_positions 함수의 char_to_token을 하면 context에서 토큰 위치를 반환. 전체 input에서 찾는걸로 바꾸어야하지않나?


추가되어야 할 것

1. 외부데이터 추가하여 똑같이 전처리하기
2. 조금씩 잘라서 학습시키는 방법 ,

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.7 MB/s eta 0:00:00


In [2]:
import torch
import sys

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from typing import List, Tuple, Dict, Any #원래랑 같음(파싱)
import json
import random

class KoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))#다수의 질문에 해당하는 지문 매핑
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        context = paragraph['context']
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        answers = qa['answers']

        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)


In [8]:
#데이터 로드 확인
dataset = KoMRC.load('/content/drive/MyDrive/Colab Notebooks/p2-korean-mrc/train.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Number of Samples: 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': [{'text': '한 달가량', 'answer_start': 478}, {'text': '한 달', 'answer_start': 478}]}


In [9]:
train_dataset, dev_dataset = KoMRC.split(dataset)#train,val 로 나눠주기
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Train Samples: 10834
Number of Dev Samples: 1203
{'guid': '844e22ab28924c1697d5ac28801b34c1', 'context': '지난해 주요 연극상을 나눠 가졌던 세 편의 작품이 올봄에 나란히 앙코르 무대를 갖는다. 대한민국연극대상 연기·무대예술상, 동아연극상 작품·희곡·연기상 등을 수상한 ‘알리바이 연대기’(17~20일 대학로 아르코예술극장 대극장, 25일~5월11일 서계동 국립극단 백성희장민호극장), 연극대상에서 대상과 희곡상을 받은 ‘여기가 집이다’(18일~5월22일 대학로 연우소극장), 연극대상 작품·연출상과 김상열연극상 수상작인 ‘황금용’(5월9~18일 서강대 메리홀 대극장)이다. 초연 당시 짧은 상연 기간과 낮은 인지도 등으로 공연을 놓친 연극팬에겐 평단으로부터 작품성을 인정받은 수작을 관람할 수 있는 기회다. ‘알리바이 연대기’는 희곡을 쓰고 연출한 김재엽의 가족사에 근거한 다큐멘터리 드라마다. 1930년에 태어난 한 개인의 사적인 연대기를 바탕으로 그 사이를 파고드는 역사적 순간들을 정밀하게 조명한다. 연출가는 “공적인 권력이 사적인 권리를 지켜주기보다 억압하기 일쑤였던 한국 현대사 속에서 개인은 언제나 무죄를 입증하며 하루하루 자신을 지켜내야 하는 ‘알리바이의 연대기’ 속에서 살아왔다”고 말한다.한국연극평론가협회는 이 작품을 ‘2013년 올해의 연극 베스트3’로 선정하며 “촘촘하고 세세하게 삶에 천착해 개인과 역사에 대한 이분법적 관점을 극복한다. 정치극에 대한 새로운 가능성을 보여줬다”고 평했다. 이 작품으로 연기상을 휩쓴 남명렬을 비롯해 지춘성 정원조 등 초연 배우들이 그대로 출연한다.‘여기가 집이다’는 허름하고 볼품 없는 ‘20년 전통’의 고시원에 모여 사는 사람들의 절망과 희망을 그린 작품. ‘차력사와 아코디언’ ‘택배 왔어요’를 만든 극단 이와삼의 장우재 대표가 직접 대본을 쓰고 연출했다. 나름의 규칙을 가지고 평화로웠던 고시원에 새로운 주인으로 등장한 ‘2

In [10]:
train_dataset[0]
#answer_start 772 뜻=  772번째 글자부터 세탁기가 있다. 우리는 모델에 넣기 위해서는 token 단위로 끊어서 토큰 단위로 셋을때 숫자로 바꾸어주어야 함.

{'guid': 'a21358b587d84173ba5a00a35e07d868',
 'context': '한국은 캐나다와 자유무역협정(FTA)을 맺으면서 ‘경제영토’를 북미 전체로 확장하게 됐다. 캐나다는 G8 회원국이자 세계 11위 경제 대국이다. 한국과의 교역 규모는 지난해 99억2200만달러로 적은 편이지만 시장의 잠재 성장 가능성은 높다는 평가다. 특히 FTA 발효 2년 후 자동차 관세가 완전 철폐되면서 한국산 자동차의 가격 경쟁력이 높아질 전망이다. ○일본 앞서 시장 선점한·캐나다 FTA 체결로 가장 이득을 보는 품목은 자동차다. 캐나다는 한국의 5대 자동차시장이다. 지난해 캐나다에 수출한 자동차 수는 13만3000대. 캐나다는 현재 한국산 자동차에 부과하는 관세 6.1%를 발효 시점부터 2년 동안 단계적으로 없애기로 했다. 이렇게 되면 일본·유럽산 자동차보다 가격경쟁력에서 앞설 수 있게 된다. 북미자유무역협정(NAFTA) 회원국인 미국·멕시코산 자동차와 비슷한 조건에서 경쟁할 수 있다. 작년 캐나다 자동차시장 점유율은 미국 44.5%, 일본 33.6%, 한국 12.0%, 유럽 9.9% 등이다. 캐나다가 일본과 FTA 협상을 진행 중이고 유럽연합(EU)과는 추가 협상 문제로 발효가 늦어지고 있어 FTA 발효를 서두른다면 적어도 수년간은 시장 선점 효과를 누릴 수 있을 것으로 보인다. 김태년 한국자동차산업협회 이사는 “소형차의 경우 영업이익률이 3%밖에 안되기 때문에 관세 철폐 효과가 클 것으로 기대된다”고 말했다.주요 수출 품목인 자동차 부품도 3년 내 6%에 달하는 관세가 사라진다. 7%에 이르는 타이어 관세는 5년 뒤 완전 철폐된다. 관세율이 8%인 세탁기는 FTA 발효 즉시 없어지고, 6%인 냉장고 관세도 3년 내 철폐돼 가전제품 수출에도 파란불이 켜졌다. 평균 관세율 5.9%인 섬유도 대부분 3년 내 관세가 사라진다. 개성공단 원산지 인정 문제에 관해서는 향후 역외가공지역위원회 설립과 충족 기준을 논의하기로 했다. 이와 관련, 박근혜 대통령은 “

In [11]:
#train, val 별 context, question, answer끼리 모읍니다. 
train_contexts,val_contexts,train_questions, train_answers,val_questions, val_answers = [],[],[],[],[],[]
for i in range(len(train_dataset)):
    train_contexts.append(train_dataset[i]['context'])
    train_questions.append(train_dataset[i]['question'])
    train_answers.append(train_dataset[i]['answers'][0])

In [12]:
for i in range(len(dev_dataset)):
    val_contexts.append(dev_dataset[i]['context'])
    val_questions.append(dev_dataset[i]['question'])
    val_answers.append(dev_dataset[i]['answers'][0])

In [13]:
print(train_contexts[0],train_questions[0],train_answers[0],sep='\n')#확인
# 본문 내에서 정답이 시작되는 음절 순서가 'answer start'에 저장되어 있습니다.
# 정답 음절 끝은 'text': '세탁기' 를 보면 알 수 있다고 함.----start에 text길이만큼 더한것 = end

한국은 캐나다와 자유무역협정(FTA)을 맺으면서 ‘경제영토’를 북미 전체로 확장하게 됐다. 캐나다는 G8 회원국이자 세계 11위 경제 대국이다. 한국과의 교역 규모는 지난해 99억2200만달러로 적은 편이지만 시장의 잠재 성장 가능성은 높다는 평가다. 특히 FTA 발효 2년 후 자동차 관세가 완전 철폐되면서 한국산 자동차의 가격 경쟁력이 높아질 전망이다. ○일본 앞서 시장 선점한·캐나다 FTA 체결로 가장 이득을 보는 품목은 자동차다. 캐나다는 한국의 5대 자동차시장이다. 지난해 캐나다에 수출한 자동차 수는 13만3000대. 캐나다는 현재 한국산 자동차에 부과하는 관세 6.1%를 발효 시점부터 2년 동안 단계적으로 없애기로 했다. 이렇게 되면 일본·유럽산 자동차보다 가격경쟁력에서 앞설 수 있게 된다. 북미자유무역협정(NAFTA) 회원국인 미국·멕시코산 자동차와 비슷한 조건에서 경쟁할 수 있다. 작년 캐나다 자동차시장 점유율은 미국 44.5%, 일본 33.6%, 한국 12.0%, 유럽 9.9% 등이다. 캐나다가 일본과 FTA 협상을 진행 중이고 유럽연합(EU)과는 추가 협상 문제로 발효가 늦어지고 있어 FTA 발효를 서두른다면 적어도 수년간은 시장 선점 효과를 누릴 수 있을 것으로 보인다. 김태년 한국자동차산업협회 이사는 “소형차의 경우 영업이익률이 3%밖에 안되기 때문에 관세 철폐 효과가 클 것으로 기대된다”고 말했다.주요 수출 품목인 자동차 부품도 3년 내 6%에 달하는 관세가 사라진다. 7%에 이르는 타이어 관세는 5년 뒤 완전 철폐된다. 관세율이 8%인 세탁기는 FTA 발효 즉시 없어지고, 6%인 냉장고 관세도 3년 내 철폐돼 가전제품 수출에도 파란불이 켜졌다. 평균 관세율 5.9%인 섬유도 대부분 3년 내 관세가 사라진다. 개성공단 원산지 인정 문제에 관해서는 향후 역외가공지역위원회 설립과 충족 기준을 논의하기로 했다. 이와 관련, 박근혜 대통령은 “개성공단 제품이 한국산으로 인정받아 관세 혜택을 받을 수 있도록 캐나다와 적극 협의해나갈 것”이라고 말했

In [14]:
# 정답에 들어가는 데이터는 start, end position.

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        # 모델 학습을 위해 정답 데이터를 만들겠습니다.
        # 정답 데이터는 start음절과 end 음절로 구성되어 있습니다.
        # 모델은 전체 토큰 중에서 start token과 end token을 찾아내는 것을 목표로 학습하게 됩니다.
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)#end글자를 명시하는과정
        
        #- 상관안쓰셔도 되는 부분(원데이터의 문데가 있던 경우가 있어서 확인하는 과정)
        # sometimes squad answers are off by a character or two – fix this 
        # 실제 본문에서 해당 음절 번호로 잘라냈을 때, 정답과 같은지 검사해서 start, end를 보정합니다 :-)
        # '이순신은 조선 중기의 무신이다' -> '이순신' -> start: 0, end: 4
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
    return answers

train_answers = add_end_idx(train_answers, train_contexts)
val_answers = add_end_idx(val_answers, val_contexts)

In [15]:
train_answers[0]#end가 추가된 것을 볼 수 있습니다. 

{'text': '세탁기', 'answer_start': 772, 'answer_end': 775}

In [16]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_questions,train_contexts, truncation=True, padding=True)#질문이 먼저가도록
val_encodings = tokenizer(val_questions,val_contexts, truncation=True, padding=True)

In [17]:
train_encodings[0]#transfomers 라이브러리의 토크나이저를 쓰면 저절로 input ids, token_type_ids, attentionmask 가 붙여집니다. 

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [18]:
print(train_encodings[0].ids,#input_ids
      train_encodings[0].type_ids,#token_type_ids
      train_encodings[0].tokens,
      train_encodings[0].offsets,
      train_encodings[0].attention_mask,
      train_encodings[0].special_tokens_mask,#cls,sep=1,나머지 =0
      train_encodings[0].overflowing,sep='\n' )

[101, 48556, 11882, 9792, 89782, 10459, 9981, 98489, 9323, 119449, 24683, 9462, 18784, 8900, 101880, 9405, 17342, 32815, 9938, 68055, 10892, 136, 102, 48556, 10892, 9792, 89782, 12638, 9651, 42815, 32537, 23160, 119435, 16605, 113, 143, 24951, 114, 9633, 9264, 80046, 12424, 100, 8885, 17730, 30858, 26444, 100, 9233, 9366, 22458, 96567, 11261, 9994, 13890, 17594, 9097, 11903, 119, 9792, 89782, 11018, 144, 11396, 9998, 14279, 20479, 55171, 32613, 10193, 19855, 8885, 17730, 9069, 20479, 11925, 119, 48556, 102003, 8907, 23160, 8922, 39420, 11018, 9706, 33305, 14523, 12187, 91837, 71793, 21069, 19105, 89851, 30873, 11261, 9664, 10892, 9924, 44359, 19105, 9485, 83685, 9655, 36210, 9434, 13890, 8843, 74986, 107442, 9028, 82034, 9926, 11287, 11903, 119, 39671, 143, 24951, 9323, 119449, 92360, 10003, 9651, 99183, 8900, 101880, 9591, 16617, 9747, 119399, 96377, 48556, 21386, 9651, 99183, 10459, 8843, 45465, 8885, 119202, 61964, 9028, 16985, 48599, 9665, 89292, 11925, 119, 1857, 18392, 40419, 953

In [ ]:
#tokenizer.decode(train_encodings[0].ids)

In [19]:
train_answers[0]['answer_start']#세탁기

772

In [20]:
train_answers[0]['answer_end']

775

In [21]:
## Find out start_position and end_position in encoded dataset
#토큰 맥스 렝스를 넘서서는 부분을 픽스 할 것
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    # 이제 음절 index를 token index와 mapping하는 작업을 해보도록 하겠습니다 :-)


    for i in range(len(answers)):

        # tokenizer의 char_to_token 함수를 호출하면 음절 숫자를 token index로 바꿔줄 수 있습니다.
        
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))#(batch_index, char_index)
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
            
       
        # 아래 부분은 truncation을 위한 과정입니다.
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length#=512

        # if end position is None, the 'char_to_token' function points to the space before the correct token - > add + 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] + 1)

        # 추가된 예외 처리, 예를들어서 tokenizer와 model input의 max_length가 512인데, start와 end position이 600과 610 이면 둘다 max_length로 변경해야함.
        # 어차피 max_length가 512인 모델은 정답을 볼 수 없음.
        # if start_positions[-1] is None or start_positions[-1] > tokenizer.model_max_length:
        #     start_positions[-1] = tokenizer.model_max_length
        
        # if end_positions[-1] is None or end_positions[-1] > tokenizer.model_max_length:
        #     end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})#encoding에 start,end 추가
    return encodings

train_encodings_changed = add_token_positions(train_encodings, train_answers)
val_encodings_changed = add_token_positions(val_encodings, val_answers)

In [23]:
print(train_encodings_changed.start_positions)#추가된것 확인 train_encodings[0] > train_encodings_changed

[512, 512, 14, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 2, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 14, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 9, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 1, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 51

In [24]:
print(train_encodings_changed.end_positions)#추가된것 확인

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 5, None, None, None, None, None, None, None, None, None, None, None, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 6, None, None, None, None, None, None, 6, None, None, None, 16, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 12, None

In [25]:
train_answers[2]['answer_start']

18

In [26]:
print(train_encodings.char_to_token(2, train_answers[2]['answer_end'] + 1))

None


In [27]:
train_answers[2]

{'text': '9월14일', 'answer_start': 18, 'answer_end': 23}

In [28]:
print(train_encodings.char_to_token(2,18))#context에서 토큰 번호인데

14


In [29]:
train_encodings[2].tokens[14]

'날'

In [30]:
train_encodings[2].tokens[31:34]

['9월', '##14', '##일']

In [31]:
#데이터가 준비되었습니다. 훈련에 쉽게 사용할 수 있도록 PyTorch/TensorFlow 데이터 세트에 넣읍시다.
# In PyTorch, we define a custom Dataset class

import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [32]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [33]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

test해보기

In [ ]:
test_dataset = KoMRC.load('/kaggle/input/7th-goorm-project-2-korean-mrc/test.json')

In [ ]:
test_dataset = TokenizedKoMRC.load('/kaggle/input/7th-goorm-project-2-korean-mrc/test.json')
test_dataset = IndexerWrappedDataset(test_dataset, indexer)
print("Number of Test Samples", len(test_dataset))
print(test_dataset[0])

In [ ]:
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/baseline.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for sample in tqdm(test_dataset, "Testing"):
        input_ids, token_type_ids = [
            torch.tensor(sample[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
    
        with torch.no_grad():
            start_logits, end_logits = model(input_ids=input_ids[None, :], token_type_ids=token_type_ids[None, :])
        start_logits.squeeze_(0), end_logits.squeeze_(0)
    
        start_prob = start_logits[token_type_ids.bool()][1:-1].softmax(-1)
        end_prob = end_logits[token_type_ids.bool()][1:-1].softmax(-1)
        probability = torch.triu(start_prob[:, None] @ end_prob[None, :])
        index = torch.argmax(probability).item()
    
        start = index // len(end_prob)
        end = index % len(end_prob)
    
        start = sample['position'][start][0]
        end = sample['position'][end][1]

        rows.append([sample["guid"], sample['context'][start:end]])
    
    writer.writerows(rows)